In [86]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
!pip install geocoder
!pip install geopy
import geocoder
from geopy.geocoders import Nominatim
import folium
from sklearn.cluster import KMeans


In [44]:
source = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(source.text, 'lxml')
data = []
columns = []
table = soup.find(class_='wikitable')
for index, tr in enumerate(table.find_all('tr')):
    section = []
    for td in tr.find_all(['th','td']):
        section.append(td.text.rstrip())
    if (index == 0):
        columns = section
    else:
        data.append(section)
        
geoDF = pd.read_csv('http://cocl.us/Geospatial_data')

In [46]:
canada_df = pd.DataFrame(data = data,columns = columns).set_index('Postal Code')
canada_df = canada_df[canada_df['Borough'] != 'Not assigned'].drop_duplicates()

geoDF =geoDF.drop_duplicates().set_index('Postal Code')

print(canada_df.head(11))

print(geoDF.head())


                      Borough                                 Neighborhood
Postal Code                                                               
M3A                North York                                    Parkwoods
M4A                North York                             Victoria Village
M5A          Downtown Toronto                    Regent Park, Harbourfront
M6A                North York             Lawrence Manor, Lawrence Heights
M7A          Downtown Toronto  Queen's Park, Ontario Provincial Government
M9A                 Etobicoke      Islington Avenue, Humber Valley Village
M1B               Scarborough                               Malvern, Rouge
M3B                North York                                    Don Mills
M4B                 East York              Parkview Hill, Woodbine Gardens
M5B          Downtown Toronto                     Garden District, Ryerson
M6B                North York                                    Glencairn
              Latitude  L

In [71]:
finalDF = pd.concat([canada_df,geoDF],axis=1, join='inner')
finalDF.index.name = 'PostalCode'
finalDF.reset_index(inplace=True)
print(finalDF.head())

  PostalCode           Borough                                 Neighborhood  \
0        M3A        North York                                    Parkwoods   
1        M4A        North York                             Victoria Village   
2        M5A  Downtown Toronto                    Regent Park, Harbourfront   
3        M6A        North York             Lawrence Manor, Lawrence Heights   
4        M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government   

    Latitude  Longitude  
0  43.753259 -79.329656  
1  43.725882 -79.315572  
2  43.654260 -79.360636  
3  43.718518 -79.464763  
4  43.662301 -79.389494  


# Part 3

In [74]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="tl-toronto-neigh")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('Toronto coords are {}, {}.'.format(latitude, longitude))


map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

for lat, long, post, borough, neigh in zip(finalDF['Latitude'], finalDF['Longitude'],  finalDF['PostalCode'], finalDF['Borough'], finalDF['Neighborhood']):
    label = "{} ({}): {}".format(borough, post, neigh)
    popup = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius=4,
        popup=popup,
        color='pink',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)
    
map_toronto

Toronto coords are 43.6534817, -79.3839347.


In [93]:
CLIENT_ID = 'G4OUIYLQ5ZJZOI1UWQI4EPH3AXSGYAW4WFT12HZS01JEEFEW' 
CLIENT_SECRET = 'OBSH2XQCZE0V55JK3JNXVZP0UBPBUDO5MUOD23BEI4WI5X2W' 
VERSION = '20190330'
num_top_venues = 10
radius = 500
LIMIT = 100

venues = []

for lat, long, post, borough, neighborhood in zip(finalDF['Latitude'], finalDF['Longitude'], finalDF['PostalCode'], finalDF['Borough'], finalDF['Neighborhood']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            post, 
            borough,
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

venues_df = pd.DataFrame(venues)
venues_df.columns = ['PostalCode', 'Borough', 'Neighborhood', 'BoroughLatitude', 'BoroughLongitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']
print(venues_df.shape)
print(venues_df.head())
venues_df.head()
venues_df.groupby(['PostalCode', 'Borough', 'Neighborhood'])['VenueName'].count()

toronto_central_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

toronto_central_onehot['PostalCode'] = venues_df['PostalCode'] 
toronto_central_onehot['Borough'] = venues_df['Borough'] 
toronto_central_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

fixed_columns = list(toronto_central_onehot.columns[-3:]) + list(toronto_central_onehot.columns[:-3])
toronto_central_onehot = toronto_central_onehot[fixed_columns]

print(toronto_central_onehot.shape)
toronto_central_onehot.head()
toronto_central_venues_freq = toronto_central_onehot.groupby(['PostalCode', 'Borough', 'Neighborhoods']).mean().reset_index()


indicators = ['st', 'nd', 'rd']

areaColumns = ['PostalCode', 'Borough', 'Neighborhoods']
freqColumns = []
for ind in np.arange(num_top_venues):
    try:
        freqColumns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        freqColumns.append('{}th Most Common Venue'.format(ind+1))
columns = areaColumns+freqColumns

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['PostalCode'] = toronto_central_venues_freq['PostalCode']
neighborhoods_venues_sorted['Borough'] = toronto_central_venues_freq['Borough']
neighborhoods_venues_sorted['Neighborhoods'] = toronto_central_venues_freq['Neighborhoods']

KeyError: 'groups'